In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from xgboost import plot_tree
import graphviz
from xgboost import to_graphviz
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from mpl_toolkits.mplot3d import Axes3D



In [ ]:

train_data = pd.read_csv('train.csv')
train_data.describe()

In [ ]:
particle_count = train_data['Label'].value_counts()

print(particle_count)

statistical_count = train_data.describe()

# print(statistical_count)




In [ ]:
plt.figure(figsize=(12, 8))
sns.set_theme(style="whitegrid")

particle_types = train_data['Label'].unique()

for particle in particle_types:
    
    subset = train_data[train_data['Label'] == particle]
    
    sns.histplot(data=subset, x='TrackP', log_scale=True, element="step", common_norm=False, fill=True, alpha=0.2, label=particle)

plt.title('Momentum Distribution for Each Particle Type', fontsize=16)
plt.xlabel('Track Momentum (TrackP) - Log Scale', fontsize=12)
plt.ylabel('Number of Particles', fontsize=12)
plt.legend(title='Particle Type')
plt.show()


In [ ]:
print("Replacing -999.0 with 0")
train_data.replace(-999.0, 0, inplace=True)
train_data.describe()

In [ ]:
y = train_data['Label']
X = train_data.drop('Label', axis=1)

print("X and y created successfully")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=73)

print("Done Splitting")

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
lab_enc = LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_test_encoded = lab_enc.transform(y_test)
print("Label mapping:")
for i, class_name in enumerate(lab_enc.classes_):
    print(f"'{class_name}' -> {i}")


In [ ]:
# rf_model = RandomForestClassifier(n_estimators=100, random_state=73, n_jobs=-1)
# rf_model.fit(X_train, y_train)
# print("Training complete")

In [ ]:
# y_pred = rf_model.predict(X_test)
# print("Predictions complete.")

# accuracy = accuracy_score(y_test, y_pred)
# print(f"\nModel Accuracy: {accuracy * 100:.2f}%")

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))


In [ ]:
xgb_model = XGBClassifier(n_estimators=100, random_state=73, use_label_encoder=False, eval_metric='mlogloss')

xgb_model.fit(X_train, y_train_encoded)
print("Training Complete")


In [ ]:
y_pred_xgb = xgb_model.predict(X_test)

accuracy_xgb = accuracy_score(y_test_encoded, y_pred_xgb)
print(f"\nXGBoost Model Accuracy: {accuracy_xgb * 100:.2f}%")
print(classification_report(y_test_encoded, y_pred_xgb, target_names=lab_enc.classes_))


In [ ]:
scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_test_scaled = scalar.fit_transform(X_test)
print("Data scaling complete")


In [ ]:
# nn_model = keras.Sequential([
#   keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
#   keras.layers.Dense(64, activation='relu'),
#   keras.layers.Dense(6, activation='softmax')
# ])
# nn_model.compile(
#     optimizer='adam',
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )
# print("Model Summary")


# nn_model.summary()

In [ ]:

importances = xgb_model.feature_importances_
feature_names = X.columns # Assumes 'X' is your DataFrame of features

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(15), palette='viridis')
plt.title('Top 15 Most Important Features for the Champion XGBoost Model', fontsize=16)
plt.xlabel('Importance Score (Higher is More Important)', fontsize=12)
plt.ylabel('Feature Name', fontsize=12)
plt.tight_layout()
plt.show()



In [ ]:

print("\n--- Generating Visual #2: Confusion Matrix ---")

cm = confusion_matrix(y_test_encoded, y_pred_xgb, labels=xgb_model.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lab_enc.classes_)

fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(ax=ax, cmap='Blues', xticks_rotation='vertical')
plt.title('Confusion Matrix for the Champion XGBoost Model', fontsize=16)
plt.show()


In [ ]:
pred_probabilities = xgb_model.predict_proba(X_test)

print(f"Probabilities generated. The shape of the output is: {pred_probabilities.shape}")

print("\nProbabilities for the first 5 particles:")
print(np.round(pred_probabilities[:5], 3))

confidence_scores = np.max(pred_probabilities, axis=1)

print("\nConfidence scores for the first 5 particles:")
print(np.round(confidence_scores[:5], 3))

anomaly_df = pd.DataFrame({
    'Original_ID': X_test.index,
    'Confidence': confidence_scores
})

most_anomalous_particles = anomaly_df.sort_values(by='Confidence', ascending=True)

print(most_anomalous_particles.head(10))

In [ ]:
top_anomaly_id = most_anomalous_particles.iloc[0]['Original_ID']


anomaly_index_in_preds = X_test.index.get_loc(top_anomaly_id)
anomaly_probs = pred_probabilities[anomaly_index_in_preds]

prob_report = pd.Series(anomaly_probs, index=lab_enc.classes_)

print("\nModel's Probability Breakdown for this Particle:")
print(prob_report.sort_values(ascending=False))

In [ ]:
top_10_anomaly_ids = most_anomalous_particles.head(10)['Original_ID']

top_10_anomaly_features = X_test.loc[top_10_anomaly_ids]

print(top_10_anomaly_features.describe())



results_df = pd.DataFrame({
    'Original_ID': X_test.index,
    'Prediction': y_pred_xgb, 
    'Confidence': confidence_scores
})

confident_muons = results_df[
    (results_df['Prediction'] == 3) & (results_df['Confidence'] > 0.999)
].head(10)

confident_muon_features = X_test.loc[confident_muons['Original_ID']]

print(confident_muon_features.describe())


In [ ]:
print("\n--- Visual 1: Muon Identification with MuonFlag ---")

# Let's use a sample of the full X_train and y_train_encoded for plotting
# Otherwise, plotting all 840k points might be slow or clunky
plot_df = pd.DataFrame(X_train_scaled, columns=X.columns) # Use scaled data for consistency
plot_df['Particle_Label'] = lab_enc.inverse_transform(y_train_encoded)

plt.figure(figsize=(8, 6))
sns.countplot(x='MuonFlag', hue='Particle_Label', data=plot_df, palette='tab10')
plt.title('Distribution of MuonFlag for Different Particles', fontsize=14)
plt.xlabel('Muon Flag (0 = No, 1 = Yes)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(title='Particle Type')
plt.tight_layout()
plt.show()


In [ ]:

particle_subset = plot_df[plot_df['Particle_Label'].isin(['Electron', 'Proton', 'Pion', 'Kaon'])]

plt.figure(figsize=(10, 8))
sns.scatterplot(x='HcalE', y='EcalE', hue='Particle_Label', data=particle_subset.sample(n=10000, random_state=42),
                alpha=0.6, s=20, palette='viridis') 

plt.xscale('log')
plt.yscale('log') 
plt.xlim(1e-3, 1e2)
plt.ylim(1e-3, 1e2)

plt.title('Calorimeter Energy Deposit (EcalE vs HcalE)', fontsize=14)
plt.xlabel('Hadronic Calorimeter Energy (HcalE)', fontsize=12)
plt.ylabel('Electromagnetic Calorimeter Energy (EcalE)', fontsize=12)
plt.legend(title='Particle Type')
plt.grid(True, which="both", ls="--", c='0.7')
plt.tight_layout()
plt.show()


In [ ]:

kp_subset = plot_df[plot_df['Particle_Label'].isin(['Kaon', 'Proton'])].sample(n=10000, random_state=42)

plt.figure(figsize=(10, 8))
sns.scatterplot(x='RICH_DLLbeKaon', y='RICH_DLLbeProton', hue='Particle_Label', data=kp_subset,
                alpha=0.6, s=20, palette={'Kaon': 'green', 'Proton': 'blue'})

plt.xlim(-5, 5) 
plt.ylim(-5, 5)

plt.axhline(0, color='grey', linestyle='--', linewidth=0.8) 
plt.axvline(0, color='grey', linestyle='--', linewidth=0.8)

plt.title('(Kaons vs. Protons)', fontsize=14)
plt.xlabel('RICH_DLLbeKaon ', fontsize=12)
plt.ylabel('RICH_DLLbeProton ', fontsize=12)
plt.legend(title='Particle Type')
plt.grid(True, which="both", ls="--", c='0.7')
plt.tight_layout()
plt.show()

In [ ]:
top_3_features = importance_df['Feature'].head(3).tolist()
print(f"Plotting the 3 most important features the model used: {top_3_features}")

X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

anomaly_ids = most_anomalous_particles.head(100)['Original_ID']
anomaly_points = X_test_scaled_df.loc[anomaly_ids]

normal_ids = X_test_scaled_df.index.difference(anomaly_ids)
normal_sample_ids = np.random.choice(normal_ids, size=5000, replace=False)
normal_points = X_test_scaled_df.loc[normal_sample_ids]

print("Generating the final 3D anomaly plot... This might take a moment.")
fig = plt.figure(figsize=(15, 12))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(normal_points[top_3_features[0]],
           normal_points[top_3_features[1]],
           normal_points[top_3_features[2]],
           c='blue',
           alpha=0.3,  
           s=5,       
           label='Normal Particles (Sample)')

ax.scatter(anomaly_points[top_3_features[0]],
           anomaly_points[top_3_features[1]],
           anomaly_points[top_3_features[2]],
           c='red',
           alpha=0.9, 
           s=30,       
           label='Anomalies')

ax.set_xlabel(top_3_features[0], fontsize=12)
ax.set_ylabel(top_3_features[1], fontsize=12)
ax.set_zlabel(top_3_features[2], fontsize=12)
ax.set_title(' Anomalies in the Space', fontsize=16)

ax.legend()
plt.show()
